# Iteration 0: Acute Stroke Unit Arrivals

## Plain English Summary

The first step in coding a `simpy` model is typically coding the arrivals to the system.  In the Acute Stroke Unit (ASU) part of the model accepts four patient type arrivals model: strokes, transient ischemic attacks (TIA; these are strokes where symptoms resolve rapidly), complex neurological, and other (medical outliers).



## Notebook imports

In [1]:
from IPython.display import Markdown, display

## LLM Prompt

In [2]:
display(Markdown("inputs/01_iteration.md"))

### Main command

Code a discrete-event simulation model of an Acute Stroke Unit (ACU) in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code.

### General model logic

All time units in the model are in days.

Each patient in the model has a unique identifier.  The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive at the ACU from four different sources: Stroke, TIA, Complex Neurological, or Other. Each source had a different inter-arrival time distribution.  After patients arrive they immediately leave the model.

All patient types must have their own generator function.

The model should print out useful information after each event.

The model should include a user settable run length. This defaults to 5 years.


### Simulation inputs

the interarrival time distributions and parameters of patients are dependent on patient type. For each distribution time is measured in days.

Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2


### simulation methodology

numpy should be use for sampling.

each interarrival distribution should have its own numpy.random.Generator object.

## Date and time of input to LLM

> 1545 9th February 2024

## Observations on the LLM Output

* The model code is adding each patient type into a simpy container after they have arrived. It is unclear exactly why it is doing this and it will be removed in iteration  in favour of tracking counts of patient types.  E.g.

```python
self.stroke_arrival = simpy.Container(env, init=0)
```

* The run length 365 * 5 in the correct units and the correct calculation of 1825

* The design to use `numpy` random number generators did not use the request `numpy.random.Generator` object and uses old cold instead (with a single stream). E.g. for the main stroke patient the sampling uses:
```python
interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
```

## Sources reported by the LLM

* https://youtu.be/qe3sms-TqD0
* https://stackoverflow.com/questions/74181096/exception-during-discrete-event-execution
* https://www.researchgate.net/publication/274084058_An_Agent-based_Discrete_Event_Simulation_Approach_for_Modeling_Large-Scale_Disaster_Evacuation_Network
* https://apps.dtic.mil/sti/tr/pdf/ADA567384.pdf

## LLM Output

In [7]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days")

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)


Stroke patient 1 arrived at 1.4940903712349 days
Stroke patient 2 arrived at 2.643664994403456 days
Stroke patient 3 arrived at 5.213488475362864 days
TIA patient 4 arrived at 5.689238960114544 days
Complex Neuro patient 5 arrived at 5.956175127209715 days
Stroke patient 6 arrived at 7.7288577403342105 days
Stroke patient 7 arrived at 8.452974479712319 days
Stroke patient 8 arrived at 9.266201023310213 days
Stroke patient 9 arrived at 10.64648152549487 days
Stroke patient 10 arrived at 10.969627933248479 days
Stroke patient 11 arrived at 11.466949736250493 days
Stroke patient 12 arrived at 11.942738362756899 days
Other patient 13 arrived at 12.398436663837552 days
TIA patient 14 arrived at 13.33171299479832 days
Stroke patient 15 arrived at 13.655560760914184 days
Complex Neuro patient 16 arrived at 13.727013433691054 days
Complex Neuro patient 17 arrived at 14.014512560423421 days
Other patient 18 arrived at 14.547498998008683 days
Stroke patient 19 arrived at 14.736540335619214 days


## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [8]:
M = 10_000_000

In [22]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(ascodeu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [23]:
test_1(M)

Stroke patient 1 arrived at 0.4838795581607259 days
Stroke patient 2 arrived at 3.744706906850387 days
Stroke patient 3 arrived at 4.163964025329207 days
Stroke patient 4 arrived at 5.239655648255207 days
Stroke patient 5 arrived at 5.575604613584572 days
Stroke patient 6 arrived at 9.813243575041877 days
Stroke patient 7 arrived at 12.818862425729032 days
Stroke patient 8 arrived at 13.096222428236457 days
Stroke patient 9 arrived at 20.716103336670344 days
Stroke patient 10 arrived at 22.760427117082767 days
Stroke patient 11 arrived at 23.30320524420276 days
Stroke patient 12 arrived at 24.025877501533827 days
Stroke patient 13 arrived at 24.053597797775815 days
Stroke patient 14 arrived at 25.545012672330717 days
Stroke patient 15 arrived at 26.109929939396036 days
Stroke patient 16 arrived at 26.719687712845257 days
Stroke patient 17 arrived at 28.26955133753957 days
Stroke patient 18 arrived at 28.93935855492546 days
Stroke patient 19 arrived at 30.667418794419127 days
Stroke pat

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [27]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [28]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [30]:
TEN_DAYS = 10

In [33]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [34]:
test_3(TEN_DAYS)

Stroke patient 1 arrived at 1.3750050766492967 days
Other patient 2 arrived at 2.2415232992589615 days
TIA patient 3 arrived at 2.864134914247426 days
Stroke patient 4 arrived at 3.1733178853382604 days
Stroke patient 5 arrived at 3.694702267843006 days
Stroke patient 6 arrived at 3.8254254288647775 days
Stroke patient 7 arrived at 4.653845012725964 days
Stroke patient 8 arrived at 4.8695417115728405 days
Complex Neuro patient 9 arrived at 5.692370088891878 days
Stroke patient 10 arrived at 6.083874502835974 days
Stroke patient 11 arrived at 6.363318429999557 days
Stroke patient 12 arrived at 7.413338907944495 days
Stroke patient 13 arrived at 7.494625464142079 days
Stroke patient 14 arrived at 8.355408458364936 days
Stroke patient 15 arrived at 8.703011331516306 days
Complex Neuro patient 16 arrived at 8.995179737422866 days
Stroke patient 17 arrived at 9.366676562094453 days
Stroke patient 18 arrived at 9.48361262741882 days
10
